## 计算图原理
### AI 框架发展
- **计算图与工业级AI**：AI框架发展近期以计算图描述神经网络，催生出工业级AI框架TensorFlow和PyTorch，同时期还有Chainer等实验框架激发设计灵感。
- **TensorFlow与PyTorch路径**：二者代表不同设计路径，PyTorch侧重灵活性易用性优先改善系统性能，TensorFlow侧重系统性能优先改善灵活性，这导致技术实现方案差异大。
- **复杂模型对框架要求**：神经网络模型趋复杂，如MOE、GAN、Attention Transformer等，需框架快速分析模型算子执行依赖、梯度计算及训练参数，以优化模型、制定调度策略、实现自动梯度计算，提高训练效率。
- **主流框架共性**：主流AI框架用计算图抽象神经网络计算，以张量这种通用数据结构理解、表达、执行模型，借助计算图形象表示AI系统化问题。 

```mermaid
flowchart TD
    style A fill:#e6f7ff,stroke:#336699,stroke-width:2px
    style B fill:#f0f9eb,stroke:#67b550,stroke-width:2px
    style C fill:#fff1e6,stroke:#d68910,stroke-width:2px
    style D fill:#f5f0ff,stroke:#9966cc,stroke-width:2px
    style E fill:#e6ffed,stroke:#38b000,stroke-width:2px
    style F fill:#fff5f0,stroke:#e74c3c,stroke-width:2px
    style G fill:#e6ffff,stroke:#009999,stroke-width:2px
    subgraph 前端
        A[前端编程语言和接口<br>Python, Lua, R, C++, JavaScript]
    end
    subgraph 统一表示
        B[神经网络模型中间表示<br>计算图]
    end
    subgraph 优化层
        C[自动微分]
        D[计算图优化]
    end
    subgraph 运行时
        E[内存管理，计算图调度 & 执行]
    end
    subgraph 底层库
        F[内核代码优化与编译<br>GPU kernel, auto kernel generation]
        G[多硬件支持<br>CPU, GPU, RDMA devices]
    end
    A --> B
    B --> C
    C --> D
    D --> E
    E --> F
    F --> G
```


神经网络训练流程有五个过程：前向计算、计算损失、自动求导、反向传播、更新模型参数，在基于计算图的 AI 框架中，这五个阶段统一由基本算子构成计算图，算子是数据流图节点，由后端高效实现。 

### 计算图构成
#### 概念
**张量**
是数学分支学科，起源于力学，是物理和力学有力工具，因其特性重要，是矢量概念推广。在几何代数中，标量是零阶张量，向量一阶，矩阵二阶。在 AI 框架中，数据用张量表示，如图像任务中一副图片是 3 维张量，自然语言处理中一个句子是 2 维张量，一组图像或多个句子增加批量维度，N 张彩色图片是四维张量。 

### 张量和张量操作

在执行计算任务中，数据常常被组织成一个高维数组，整个计算任务的绝大部分时间都消耗在高维数组上的数值计算操作上。高维数组和高维数组之上的数值计算是神经网络的核心，构成了计算图中最重要的一类基本算子。在 AI 框架的数据中主要有稠密张量和稀疏张量，这里先考虑最为常用的稠密张量。

张量作为高维数组，是对标量，向量，矩阵的推广。AI 框架对张量的表示主要有以下几个重要因素：

1. **元素数据类型**：在一个张量中，所有元素具有相同的数据类型，如整型，浮点型，布尔型，字符型等数据类型格式；
2. **形状**：张量每个维度具有固定的大小，其形状是一个整型数的元组，描述了一个张量的维度以及每个维度的长度；
3. **设备**：决定了张量的存储设备，如在通用处理器 CPU 中的 DDR 上还是 GPU/NPU 的 HBM 上等。

轴一般按照从全局到局部的顺序进行排序：首先是批次轴，随后是空间维度，最后是每个位置的特征。这样，在内存中，特征向量就会位于连续的区域。例如针对形状为 (3, 2, 4, 5) 的四维张量进行表示，其内存表示如图中右侧所示。

**高维数组的优势**
- 为开发者提供逻辑上易于理解的方式组织同质数据，提高编程可理解性。
- 便于后端自动推断并完成元素逻辑与物理存储空间的映射。
- 张量操作隐含高数据并行性，适合在 SIMD 并行后端上加速。（SIMD：单指令多数据 Single Instruction Multiple Data）

### 计算图表示

计算图是用来描述运算的有向无环图，有两个主要元素：节点 (Node) 和边 (Edge)。节点表示数据，如向量、矩阵、张量；边表示具体执行的运算，如加、减、乘、除和卷积等。

在 AI 框架中会稍微有点不同，其计算图的基本组成有两个主要的元素：1）基本数据结构张量和 2）基本计算单元算子。节点代表 Operator 具体的计算操作（即算子），边代表 Tensor 张量。整个计算图能够有效地表达神经网络模型的计算逻辑和状态。


- **基本数据结构张量**：张量通过 `shape` 来表示张量的具体形状，决定在内存中的元素大小和元素组成的具体形状；其元素类型决定了内存中每个元素所占用的字节数和实际的内存空间大小

- **基本运算单元算子**：具体在加速器 GPU/NPU 中执行运算的是由最基本的代数算子组成，另外还会根据深度学习结构组成复杂算子。每个算子接受的输入输出不同，如 Conv 算子接受 3 个输入 Tensor，1 个输出 Tensor

### Pytorch 计算图

在 Pytorch 的计算图中，同样由节点和边组成，节点表示张量或者函数，边表示张量和函数之间的依赖关系。其中 Pytorch 中的计算图是动态图。这里的动态主要有两重含义。

- 第一层含义是：计算图的正向传播是立即执行的。无需等待完整的计算图创建完毕，每条语句都会在计算图中动态添加节点和边，并立即执行正向传播得到计算结果。


In [7]:
import torch
w = torch.tensor([[3.0,1.0]],requires_grad=True)
b = torch.tensor([[3.0]],requires_grad=True)
X = torch.randn(10,2)
Y = torch.randn(10,1)

# Y_hat 定义后其正向传播被立即执行，与其后面的 loss 创建语句无关
Y_hat = X@w.t() + b
print(Y_hat.data)

loss = torch.mean(torch.pow(Y_hat-Y,2))
print(loss.data)

tensor([[ 3.1854],
        [ 7.6176],
        [ 3.7915],
        [ 1.6255],
        [ 2.5597],
        [ 7.4115],
        [ 3.7226],
        [-0.9073],
        [ 2.3005],
        [-1.9494]])
tensor(19.1457)



- 第二层含义是：计算图在反向传播后立即销毁。下次调用需要重新构建计算图。如果在程序中使用了 backward 方法执行了反向传播，或者利用 torch.autograd.grad 

In [6]:
# 如果再次执行反向传播将报错
loss.backward(retain_graph = True)  

# 计算图在反向传播后立即销毁，如果需要保留计算图, 需要设置 retain_graph = True
loss.backward(retain_graph = True) 

### 计算图中 Function

计算图中的另外一种节点是 Function, 实际上为对张量操作的函数，其特点为同时包括正向计算逻辑和反向传播的逻辑。通过继承 torch.autograd.Function 来创建。

以创建一个 ReLU 函数为例：

In [8]:
class MyReLU(torch.autograd.Function):

    # 正向传播逻辑，可以用 ctx 存输入张量，供反向传播使用
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        # clamp 函数将输入张量中的负数全部置为 0
        return input.clamp(min=0)

    #反向传播逻辑
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input
# relu 现在也可以具有正向传播和反向传播功能
relu = MyReLU.apply
Y_hat = relu(X@w.t() + b)

loss = torch.mean(torch.pow(Y_hat-Y,2))
loss.backward()

print(w.grad)
print(b.grad)
print(Y_hat.grad_fn)

tensor([[3.3663, 3.5566]])
tensor([[6.7944]])


### 总结
- 计算图是 AI 框架中用于抽象和表达神经网络计算的关键数据结构，它将复杂的神经网络模型表示为一系列有向节点和边，便于自动微分和优化执行。

- 计算图由标量、向量、矩阵和张量等基本数据结构组成，并通过张量操作来执行数值计算，其中张量是多维数组的推广，适用于表达和处理数据并行性。

- PyTorch 中的计算图是动态的，正向传播即时执行，反向传播后计算图销毁，支持通过自定义 Function 来扩展新的操作，结合正向和反向逻辑实现自动微分。